<a href="https://colab.research.google.com/github/naot97/Implement_Machine_Learning/blob/master/Adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split 


In [44]:
df = pd.read_csv('train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [45]:
y = df['Survived'].values
df = df.drop(['Survived','PassengerId','Name','Ticket','Cabin'], axis = 1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [46]:
df.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [47]:
my_imputer = SimpleImputer(strategy = 'median')
df['Age'] = pd.DataFrame(my_imputer.fit_transform(df[['Age']]))

In [48]:
my_imputer = SimpleImputer(strategy = 'most_frequent')
df['Embarked'] = pd.DataFrame(my_imputer.fit_transform(df[['Embarked']]))


In [49]:
ordinal_encoder = OrdinalEncoder()
df[['Sex', 'Embarked']] = ordinal_encoder.fit_transform(df[[ 'Embarked', 'Embarked']])

In [50]:
X = df.drop(['Age','Fare'], axis = 1)
X.head()

,Pclass,Sex,SibSp,Parch,Embarked
0,3,2.0,1,0,2.0
1,1,0.0,1,0,0.0
2,3,2.0,0,0,2.0
3,1,2.0,1,0,2.0
4,3,2.0,0,0,2.0


In [59]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y ,test_size = 0.08, random_state = 0)


In [92]:
M = 5
n_train, n_valid = len(X_train), len(X_valid)
w = np.ones(n_train) / n_train

In [93]:
list_clf = []
list_alpha = []
min_error = 999
min_m = -1
sum_alpha = [0 for i in range(n_valid)]
for m in range(M):
  clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)
  clf_tree.fit(X_train,y_train, sample_weight = w)
  list_clf.append(clf_tree)

  preds_train = clf_tree.predict(X_train)  
  miss = [1 if preds_train[i] != y_train[i] else 0 for i in range(n_train)]
  error = np.dot(w, miss) / sum(w)
  alpha = 0.5 * np.log( (1 - error) / float(error))

  miss_to_update = [1 if preds_train[i] != y_train[i] else -1 for i in range(n_train)]
  w = np.multiply(w, np.exp([float(i) * alpha for i in miss_to_update]))
  list_alpha.append(alpha)

  valid_preds_m = clf_tree.predict(X_valid)  
  sum_alpha = [sum(j) for j in zip(sum_alpha, [alpha if valid_preds_m[i] == 1 else -alpha for i in range(n_valid)])] 


  valid_preds = [1 if sum_alpha[i] > 0 else 0 for i in range(n_valid)]
  miss_valid =  [1 if valid_preds[i] != y_valid[i] else 0 for i in range(n_valid)]
  valid_error = sum(miss_valid)/n_valid
  print(f"Espoch {m} - Valid error:", valid_error)
  if valid_error < min_error:
    min_error = valid_error
    min_m = m

Espoch 0 - Valid error: 0.25
Espoch 1 - Valid error: 0.25
Espoch 2 - Valid error: 0.2916666666666667
Espoch 3 - Valid error: 0.2916666666666667
Espoch 4 - Valid error: 0.2916666666666667
